In [ ]:
# import os
# import json
# from tqdm import tqdm
# import random

# # ------------------------------------------------------------------
# # Roots
# # ------------------------------------------------------------------
# geom_root = "/content/drive/MyDrive/1_Diamond/CADVLM/Omni-CAD-subset-complete/json"
# txt_root  = "/content/drive/MyDrive/1_Diamond/CADVLM/Omni-CAD-subset-complete/txt"
# img_root  = "/content/drive/MyDrive/1_Diamond/CADVLM/Omni-CAD-subset-complete/img"
# pts_root  = "/content/drive/MyDrive/1_Diamond/CADVLM/Omni-CAD-subset-complete/pointcloud"

# # how many rendered views per shape? adjust if your dataset uses a different number
# NUM_VIEWS = 8

# # ------------------------------------------------------------------
# # Deterministic path builders (no directory listing needed)
# # ------------------------------------------------------------------
# def get_image_paths_for_shape(img_root, category, shape_id, num_views=NUM_VIEWS):
#     """
#     Build image paths using the known naming convention:
#     img_root/category/shape_id_000.png ... _00{num_views-1}.png

#     Only returns paths that actually exist.
#     """
#     cat_dir = os.path.join(img_root, category)
#     if not os.path.isdir(cat_dir):
#         return []

#     paths = []
#     for i in range(num_views):
#         fname = f"{shape_id}_{i:03d}.png"
#         path = os.path.join(cat_dir, fname)
#         if os.path.exists(path):
#             paths.append(path)

#     return paths


# def get_pointcloud_paths_for_shape(pts_root, category, shape_id):
#     """
#     Build pointcloud path using known naming convention:
#     pointcloud/category/shape_id.ext

#     We try a few typical extensions in a fixed order.
#     """
#     cat_dir = os.path.join(pts_root, category)
#     if not os.path.isdir(cat_dir):
#         return []

#     exts = [".npz", ".npy", ".ply", ".pcd"]
#     paths = []
#     for ext in exts:
#         candidate = os.path.join(cat_dir, shape_id + ext)
#         if os.path.exists(candidate):
#             paths.append(candidate)
#             # usually there's just one, but keeping a list for consistency
#     return paths


# # ------------------------------------------------------------------
# # Main builder
# # ------------------------------------------------------------------
# def build_omnicad_multimodal_pairs(geom_root, txt_root, img_root, pts_root,
#                                    max_pairs=None):
#     """
#     Build a list of examples where each example is:
#       {
#         "id":               "0003/00038800_00007",
#         "category":         "0003",
#         "shape_id":         "00038800_00007",
#         "description":      <natural language text>,
#         "json_text":        <raw Omni-CAD JSON string>,
#         "image_paths":      [list of image file paths],
#         "pointcloud_paths": [list of point cloud file paths],
#       }

#     Here we assume strict naming conventions and avoid any directory scanning
#     for images / pointclouds.
#     """

#     # ---- 1. Build caption index: id -> description ----
#     caption_by_id = {}

#     txt_files = [f for f in os.listdir(txt_root) if f.endswith(".json")]
#     txt_files = sorted(txt_files)

#     print(f"Found {len(txt_files)} caption files under txt_root")

#     for fname in tqdm(txt_files, desc="Reading caption files"):
#         path = os.path.join(txt_root, fname)
#         with open(path, "r") as f:
#             arr = json.load(f)

#         # each element like: {"id": "0003/00038800_00007", "text caption": "..."}
#         for entry in arr:
#             sid = entry.get("id")
#             if sid is None:
#                 continue

#             caption = (
#                 entry.get("text caption")
#                 or entry.get("caption")
#                 or entry.get("text")
#             )
#             if caption is None:
#                 continue

#             caption_by_id[sid] = caption

#     print(f"Collected {len(caption_by_id)} unique ids with captions")

#     # ---- 2. For each id, construct paths directly from naming scheme ----
#     pairs = []
#     for sid, caption in tqdm(
#         caption_by_id.items(),
#         total=len(caption_by_id),
#         desc="Building multimodal pairs"
#     ):
#         # sid like "0000/00000071_00005"
#         try:
#             cat, shape = sid.split("/")
#         except ValueError:
#             # unexpected id format, skip
#             continue

#         # geometry path: json_root/cat/shape.json
#         geom_path = os.path.join(geom_root, cat, shape + ".json")
#         if not os.path.exists(geom_path):
#             # geometry missing, skip
#             continue

#         # read raw JSON string (faster than parse+re-dump)
#         with open(geom_path, "r") as f:
#             json_text = f.read()

#         # images & pointclouds: constructed from known naming convention
#         image_paths = get_image_paths_for_shape(img_root, cat, shape)
#         pointcloud_paths = get_pointcloud_paths_for_shape(pts_root, cat, shape)

#         pairs.append({
#             "id": sid,
#             "category": cat,
#             "shape_id": shape,
#             "description": caption,
#             "json_text": json_text,
#             "image_paths": image_paths,
#             "pointcloud_paths": pointcloud_paths,
#         })

#         if max_pairs is not None and len(pairs) >= max_pairs:
#             break

#     print(f"Total multimodal examples: {len(pairs)}")
#     return pairs

# # ------------------------------------------------------------------
# # Run it
# # ------------------------------------------------------------------
# pairs = build_omnicad_multimodal_pairs(
#     geom_root, txt_root, img_root, pts_root,
#     max_pairs=None   # or a small number (e.g. 1000) for a test
# )

# if pairs:
#     ex = random.choice(pairs)
#     print("\nExample id:", ex["id"])
#     print("Category:", ex["category"], "Shape:", ex["shape_id"])
#     print("Description:", ex["description"])
#     print("JSON snippet:", ex["json_text"][:200])
#     print("Images:", ex["image_paths"][:3])
#     print("Pointclouds:", ex["pointcloud_paths"][:3])


In [ ]:
# out_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.jsonl"
# with open(out_path, "w") as f:
#   for ex in pairs:
#       f.write(json.dumps(ex) + "\n")
# print("Saved pairs to:", out_path)

In [ ]:
# import json
# from torch.utils.data import Dataset
# from PIL import Image
# import numpy as np

# class OmniCADPairsDataset(Dataset):
#     def __init__(
#         self,
#         jsonl_path,
#         load_images=False,
#         image_transform=None,
#         load_pointclouds=False,
#     ):
#         """
#         jsonl_path: path to omnicad_multimodal_pairs.jsonl
#         load_images: whether to load the first image as a PIL / transformed tensor
#         image_transform: torchvision-style transform for images
#         load_pointclouds: whether to load the first point cloud (npz/npy)
#         """
#         self.jsonl_path = jsonl_path
#         self.load_images = load_images
#         self.image_transform = image_transform
#         self.load_pointclouds = load_pointclouds

#         self.examples = []
#         with open(jsonl_path, "r", encoding="utf-8") as f:
#             for line in f:
#                 line = line.strip()
#                 if not line:
#                     continue
#                 self.examples.append(json.loads(line))

#         print(f"Loaded {len(self.examples)} examples from {jsonl_path}")

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, idx):
#         ex = self.examples[idx]

#         # core info: always there
#         item = {
#             "id": ex["id"],
#             "category": ex["category"],
#             "shape_id": ex["shape_id"],
#             "description": ex["description"],     # NL text
#             "json_text": ex["json_text"],         # Omni-CAD JSON as string
#             "image_paths": ex.get("image_paths", []),
#             "pointcloud_paths": ex.get("pointcloud_paths", []),
#         }

#         # --- optionally load first image ---
#         if self.load_images and item["image_paths"]:
#             img_path = item["image_paths"][0]
#             img = Image.open(img_path).convert("RGB")
#             if self.image_transform is not None:
#                 img = self.image_transform(img)
#             item["image"] = img  # only add key if we actually have an image

#         # --- optionally load first point cloud ---
#         if self.load_pointclouds and item["pointcloud_paths"]:
#             pc_path = item["pointcloud_paths"][0]
#             pts = None
#             if pc_path.endswith(".npz"):
#                 npz = np.load(pc_path)
#                 if "points" in npz:
#                     pts = npz["points"]
#                 else:
#                     pts = list(npz.values())[0]
#             elif pc_path.endswith(".npy"):
#                 pts = np.load(pc_path)
#             # you can extend this for .ply/.pcd
#             item["points"] = pts  # only add if actually loaded

#         return item


In [ ]:
# from torch.utils.data import DataLoader

# jsonl_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.jsonl"

# dataset = OmniCADPairsDataset(
#     jsonl_path,
#     load_images=False,
#     image_transform=None,
#     load_pointclouds=False,
# )

# loader = DataLoader(dataset, batch_size=4, shuffle=True)

# batch = next(iter(loader))
# print(batch["description"][0])
# print(batch["json_text"][0][:200])
# print(batch["image_paths"][0])
# print(batch["pointcloud_paths"][0])


In [ ]:
# import torch

# save_pt_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.pt"
# torch.save(dataset.examples, save_pt_path)
# print("Saved examples to:", save_pt_path)


In [ ]:
# import os
# import json
# import torch
# import numpy as np
# from PIL import Image
# import matplotlib.pyplot as plt
# import math

# # 1. Load saved examples
# save_pt_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.pt"
# examples = torch.load(save_pt_path)
# print("Loaded examples:", len(examples))

# # 2. Pick an example that has both images and point clouds, if possible
# idx = None
# for i, ex in enumerate(examples):
#     if ex.get("image_paths") and ex.get("pointcloud_paths"):
#         idx = i
#         break

# if idx is None:
#     print("No example with both image and pointcloud found; using first example.")
#     idx = 0

# ex = examples[idx]
# print("Chosen example index:", idx)
# print("ID:", ex["id"])
# print("Category:", ex["category"], "Shape ID:", ex["shape_id"])

# # 3. Print the text description
# print("\n=== Description ===")
# print(ex["description"])

# # 4. Print the CAD JSON (pretty)
# print("\n=== CAD JSON ===")
# try:
#     cad_obj = json.loads(ex["json_text"])
#     print(json.dumps(cad_obj, indent=2))
# except Exception as e:
#     print("[WARN] Could not parse json_text, printing raw string.")
#     print(ex["json_text"])

# # 5. Plot images (up to 4)
# img_paths = ex.get("image_paths", [])
# if img_paths:
#     n_imgs = len(img_paths)
#     print(f"\nShowing all {n_imgs} image(s):")
#     for p in img_paths:
#         print("  ", p)

#     # choose a grid: up to 6 columns, as many rows as needed
#     cols = min(6, n_imgs)
#     rows = math.ceil(n_imgs / cols)

#     fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
#     axes = np.array(axes).reshape(-1)  # flatten in case of 2D axes array

#     for ax, p in zip(axes[:n_imgs], img_paths):
#         img = Image.open(p).convert("RGB")
#         ax.imshow(img)
#         ax.set_title(os.path.basename(p), fontsize=8)
#         ax.axis("off")

#     # turn off any extra empty axes if rows*cols > n_imgs
#     for ax in axes[n_imgs:]:
#         ax.axis("off")

#     plt.tight_layout()
#     plt.show()
# else:
#     print("\n[INFO] No image_paths in this example.")

# # 6. Load and plot first point cloud
# pc_paths = ex.get("pointcloud_paths", [])
# if pc_paths:
#     pc_path = pc_paths[0]
#     print("\nPoint cloud path:", pc_path)

#     pts = None
#     if pc_path.endswith(".npz"):
#         npz = np.load(pc_path)
#         if "points" in npz:
#             pts = npz["points"]
#         else:
#             # fall back to first array in the npz
#             pts = list(npz.values())[0]
#     elif pc_path.endswith(".npy"):
#         pts = np.load(pc_path)
#     else:
#         print("[WARN] Unsupported point cloud format for plotting:", pc_path)

#     if pts is not None:
#         # optional subsampling for speed/visibility
#         if pts.shape[0] > 5000:
#             sel = np.random.choice(pts.shape[0], 5000, replace=False)
#             pts_plot = pts[sel]
#         else:
#             pts_plot = pts

#         fig = plt.figure(figsize=(6, 6))
#         ax = fig.add_subplot(111, projection="3d")
#         ax.scatter(pts_plot[:, 0], pts_plot[:, 1], pts_plot[:, 2], s=1)
#         ax.set_xlabel("X")
#         ax.set_ylabel("Y")
#         ax.set_zlabel("Z")
#         ax.set_title(os.path.basename(pc_path))
#         plt.tight_layout()
#         plt.show()
#     else:
#         print("[WARN] Could not load point cloud data.")
# else:
#     print("\n[INFO] No pointcloud_paths in this example.")


In [ ]:
!git clone https://github.com/NeuralCarver/Michelangelo.git

In [ ]:
!pip install omegaconf einops
!pip install pytorch-lightning

In [ ]:
!pip install -q "transformers>=4.45.0" accelerate bitsandbytes sentencepiece safetensors

In [ ]:
%cd /content/Michelangelo

In [ ]:
import torch
import torch.nn as nn
from omegaconf import OmegaConf

# Assuming this import still works and the michelangelo folder is accessible
from  michelangelo.models.tsal.sal_perceiver import AlignedShapeLatentPerceiver

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

ENCODER_SD_PATH  = "/content/drive/MyDrive/1_Diamond/CADVLM/Projects/michelangelo_point_encoder_state_dict.pt"
ENCODER_CFG_PATH = "/content/drive/MyDrive/1_Diamond/CADVLM/Projects/michelangelo_point_encoder_cfg.yaml"

# 1) Load the tiny shape-module config
shape_cfg = OmegaConf.load(ENCODER_CFG_PATH)
shape_params = dict(shape_cfg.params)  # OmegaConf -> plain dict

# 2) Instantiate the encoder directly (no Lightning, no CLIP)
# FIX: Explicitly pass device and dtype required by the constructor
michelangelo_encoder = AlignedShapeLatentPerceiver(
    **shape_params,
    device=device,
    dtype=torch.float32  # Using standard float precision
)

michelangelo_encoder.eval()

# 3) Load the saved weights
# NOTE: The model is already on 'device' from step 2
state_dict = torch.load(ENCODER_SD_PATH, map_location=device)
michelangelo_encoder.load_state_dict(state_dict, strict=True)

print("✅ Loaded standalone Michelangelo point encoder")

class MichelangeloPointTokens(nn.Module):
    """
    Wraps Michelangelo encoder + PointProjector to produce
    [B, 1, d_model] tokens suitable for feeding into LLaVA.
    """
    def __init__(self, base_encoder, projector, freeze_base: bool = True):
        super().__init__()
        self.base_encoder = base_encoder
        self.projector = projector

        if freeze_base:
            for p in self.base_encoder.parameters():
                p.requires_grad = False

    def forward(self, points, feats):
        """
        points: [B, N, 3]
        feats:  [B, N, F] (per-point features; can be zeros)
        returns: pc_tokens [B, 1, d_model]
        """
        with torch.no_grad():
            shape_embed, kl_embed, posterior = self.base_encoder.encode(
                points,
                feats,
                sample_posterior=False,
            )
            # shape_embed: [B, 768]

        pc_vec = self.projector(shape_embed)  # [B, d_model]
        pc_tokens = pc_vec.unsqueeze(1)       # [B, 1, d_model]
        return pc_tokens


In [ ]:
import torch
import torch.nn as nn

class PointProjector(nn.Module):
    """
    Projects a global point-cloud latent (e.g. Michelangelo shape_embed)
    into the LLaVA / LLM hidden dimension.

    in_dim:  dimensionality of shape_embed (768 for Michelangelo)
    out_dim: LLaVA hidden size (llava.config.hidden_size)
    hidden_dim: if not None, use a 2-layer MLP; else, a single Linear.
    """
    def __init__(self, in_dim: int, out_dim: int, hidden_dim: int = None):
        super().__init__()
        if hidden_dim is None:
            # simplest: direct linear projection
            self.proj = nn.Linear(in_dim, out_dim)
        else:
            # slightly richer: MLP with GELU
            self.proj = nn.Sequential(
                nn.Linear(in_dim, hidden_dim),
                nn.GELU(),
                nn.Linear(hidden_dim, out_dim),
            )

    def forward(self, x):
        """
        x: [B, in_dim] (e.g., shape_embed)
        returns: [B, out_dim]
        """
        return self.proj(x)


In [ ]:
import torch
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model_id = "Qwen/Qwen2-VL-2B-Instruct"

processor = AutoProcessor.from_pretrained(model_id)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
)
model.eval()

print("Loaded Qwen2-VL model")


In [ ]:
if hasattr(model.config, "text_config"):
    d_model = model.config.text_config.hidden_size
else:
    d_model = model.config.hidden_size

print("Qwen2-VL hidden size:", d_model)


In [ ]:
d_pc = 768

point_projector = PointProjector(in_dim=d_pc, out_dim=d_model).to(device)
pc_token_encoder = MichelangeloPointTokens(
    base_encoder= michelangelo_encoder,
    projector=point_projector,
    freeze_base=True,   # unfreeze later if you want
).to(device)

# test
B, N = 2, 2048
point_feats = shape_params.get("point_feats", 3)
dummy_points = torch.randn(B, N, 3, device=device)
dummy_feats  = torch.zeros(B, N, point_feats, device=device)

with torch.no_grad():
    pc_tokens = pc_token_encoder(dummy_points, dummy_feats)

print("pc_tokens shape:", pc_tokens.shape)  # should be [2, 1, d_model]


In [ ]:
tokenizer = processor.tokenizer
tokenizer.pad_token = tokenizer.eos_token  # safety
model.config.use_cache = False


In [ ]:
import json
from torch.utils.data import Dataset, random_split

jsonl_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.jsonl"

class OmniCADMultimodalJSONDataset(Dataset):
    def __init__(self, jsonl_path, max_samples=None):
        self.examples = []
        with open(jsonl_path, "r") as f:
            for i, line in enumerate(f):
                if max_samples is not None and i >= max_samples:
                    break
                ex = json.loads(line)

                desc      = ex.get("description")
                json_text = ex.get("json_text")
                pc_paths  = ex.get("pointcloud_paths", [])

                if desc is None or json_text is None or not pc_paths:
                    continue

                self.examples.append({
                    "description": desc,
                    "json_text":   json_text,
                    "point_paths": pc_paths,
                })

        print(f"Loaded {len(self.examples)} clean examples from", jsonl_path)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

full_dataset = OmniCADMultimodalJSONDataset(jsonl_path, max_samples=None)

train_ratio = 0.9
n_total = len(full_dataset)
n_train = int(train_ratio * n_total)
n_val   = n_total - n_train

train_dataset, val_dataset = random_split(full_dataset, [n_train, n_val])
print("train_dataset size:", len(train_dataset))
print("val_dataset size:  ", len(val_dataset))


In [ ]:
import torch.nn as nn

class MichelangeloMultiViewTokens(nn.Module):
    """
    points: [B, V, N, 3]
    feats:  [B, V, N, F]
    view_mask: [B, V] boolean (True for real views)
    returns: pc_tokens [B, 1, d_model]
    """
    def __init__(self, base_encoder, projector, freeze_base=True):
        super().__init__()
        self.base_encoder = base_encoder
        self.projector = projector

        if freeze_base:
            for p in self.base_encoder.parameters():
                p.requires_grad = False

    def forward(self, points, feats, view_mask=None):
        B, V, N, _ = points.shape
        device = points.device

        # flatten view dimension into batch
        pts_flat   = points.view(B * V, N, 3)
        feats_flat = feats.view(B * V, N, feats.shape[-1])

        with torch.no_grad():
            shape_embed, kl_embed, posterior = self.base_encoder.encode(
                pts_flat,
                feats_flat,
                sample_posterior=False
            )
            # shape_embed: [B*V, d_pc]

        d_pc = shape_embed.shape[-1]
        shape_embed = shape_embed.view(B, V, d_pc)   # [B, V, d_pc]

        if view_mask is not None:
            mask = view_mask.to(shape_embed.device).unsqueeze(-1)   # [B, V, 1] bool
            shape_embed = shape_embed.masked_fill(~mask, 0.0)
            denom = mask.sum(dim=1).clamp(min=1).float()           # [B, 1, 1]
            pooled = shape_embed.sum(dim=1) / denom.squeeze(-1)    # [B, d_pc]
        else:
            pooled = shape_embed.mean(dim=1)                        # [B, d_pc]

        pc_vec    = self.projector(pooled)           # [B, d_model]
        pc_tokens = pc_vec.unsqueeze(1)              # [B, 1, d_model]
        return pc_tokens


In [ ]:
pc_token_encoder = MichelangeloMultiViewTokens(
    base_encoder=michelangelo_encoder,
    projector=point_projector,
    freeze_base=True,
).to(device)


In [ ]:
import json
from torch.utils.data import Dataset

class OmniCADTriModalJSONDataset(Dataset):
    """
    Each item:
      {
        "description": <text>,
        "json_text":   <CAD JSON>,
        "image_paths": [img0, img1, ...],
        "point_paths": [pc0, ...]   # in your case usually length 1
      }
    """
    def __init__(self, jsonl_path, max_samples=None,
                 require_image=True, require_pointcloud=True):
        self.examples = []
        with open(jsonl_path, "r") as f:
            for i, line in enumerate(f):
                if max_samples is not None and i >= max_samples:
                    break
                ex = json.loads(line)

                desc      = ex.get("description")
                json_text = ex.get("json_text")
                img_paths = ex.get("image_paths", [])
                pc_paths  = ex.get("pointcloud_paths", [])

                if desc is None or json_text is None:
                    continue
                if require_image and not img_paths:
                    continue
                if require_pointcloud and not pc_paths:
                    continue

                self.examples.append({
                    "description": desc,
                    "json_text":   json_text,
                    "image_paths": img_paths,
                    "point_paths": pc_paths,
                })

        print(f"Loaded {len(self.examples)} tri-modal examples from {jsonl_path}")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]


In [ ]:
jsonl_path = "/content/drive/MyDrive/1_Diamond/CADVLM/omnicad_multimodal_pairs.jsonl"
full_dataset = OmniCADTriModalJSONDataset(jsonl_path)

from torch.utils.data import random_split

train_ratio = 0.9
n_total = len(full_dataset)
n_train = int(train_ratio * n_total)
n_val   = n_total - n_train

train_dataset, val_dataset = random_split(full_dataset, [n_train, n_val])
print("train_dataset size:", len(train_dataset))
print("val_dataset size:  ", len(val_dataset))


In [ ]:
import os
import random
import torch
import numpy as np
from PIL import Image

N_POINTS    = 2048
POINT_FEATS = 3

train_prompt_template = (
    "You are a CAD program generator.\n"
    "Given a natural language description of a CAD part, "
    "generate a valid Omni-CAD JSON program that describes the part.\n\n"
    "Description:\n{description}\n\nJSON:\n{json_text}\n"
)

def load_pointcloud_simple(path, n_points=N_POINTS):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".npz":
        data = np.load(path)
        if "points" in data:
            pts = data["points"]
        elif "pos" in data:
            pts = data["pos"]
        else:
            pts = list(data.values())[0]
    elif ext == ".npy":
        pts = np.load(path)
    else:
        raise NotImplementedError(f"Unsupported pointcloud format: {ext}")

    if pts.shape[1] > 3:
        pts = pts[:, :3]

    if pts.shape[0] >= n_points:
        idx = np.random.choice(pts.shape[0], n_points, replace=False)
        pts = pts[idx]
    else:
        pad = np.zeros((n_points - pts.shape[0], 3), dtype=pts.dtype)
        pts = np.concatenate([pts, pad], axis=0)

    return pts.astype(np.float32)


class CADTriModalCollatorRandomView:
    """
    Tri-modal collator:
      - text: description + ground-truth JSON
      - image: **ONE random view** per shape
      - pointcloud: one pc per shape

    This is lighter than multi-view but still tri-modal.
    """
    def __init__(self, processor, n_points=N_POINTS, point_feats=POINT_FEATS,
                 max_length=1024):
        self.processor   = processor             # Qwen2-VL processor
        self.tokenizer   = processor.tokenizer
        self.n_points    = n_points
        self.point_feats = point_feats
        self.max_length  = max_length

    def __call__(self, batch):
        batch_chat_prompts = []
        batch_images       = []   # one PIL image per sample
        pc_points_list     = []
        pc_feats_list      = []
        view_masks_list    = []

        for ex in batch:
            desc        = ex["description"]
            json_text   = ex["json_text"]
            image_paths = ex["image_paths"]
            point_paths = ex["point_paths"]

            # ---- text: description + GT JSON ----
            text = train_prompt_template.format(
                description=desc,
                json_text=json_text,
            )

            # one image token in the chat, because we sample ONE view
            messages = [{
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": text},
                ],
            }]

            chat_prompt = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False,
            )
            batch_chat_prompts.append(chat_prompt)

            # ---- ONE random image view ----
            if len(image_paths) == 0:
                raise ValueError("Example has no image_paths")

            img_path = random.choice(image_paths)
            img = Image.open(img_path).convert("RGB")
            batch_images.append(img)

            # ---- ONE pointcloud ----
            if len(point_paths) == 0:
                raise ValueError("Example has no point_paths")

            pc_path = point_paths[0]
            pts = load_pointcloud_simple(pc_path, n_points=self.n_points)  # [N,3]
            pts = torch.from_numpy(pts)  # [N,3]

            pts = pts.unsqueeze(0)  # [V=1,N,3]
            feats = torch.zeros(1, self.n_points, self.point_feats, dtype=torch.float32)
            view_mask = torch.ones(1, dtype=torch.bool)  # one valid pc view

            pc_points_list.append(pts)
            pc_feats_list.append(feats)
            view_masks_list.append(view_mask)

        # ---- Qwen2-VL processor: text + single image per sample ----
        proc_inputs = self.processor(
            text=batch_chat_prompts,
            images=batch_images,          # [B] list of PIL images
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=self.max_length,
        )

        input_ids      = proc_inputs["input_ids"]
        attention_mask = proc_inputs["attention_mask"]
        labels         = input_ids.clone()   # predict every token for now

        # stack pc tensors -> [B,1,N,3], [B,1,N,F], [B,1]
        points    = torch.stack(pc_points_list,  dim=0)
        feats     = torch.stack(pc_feats_list,   dim=0)
        view_mask = torch.stack(view_masks_list, dim=0)

        batch_out = {
            "input_ids":      input_ids,
            "attention_mask": attention_mask,
            "labels":         labels,
            "points":         points,
            "feats":          feats,
            "view_mask":      view_mask,
        }

        # add vision kwargs (pixel_values, image_grid_thw, etc.)
        for k, v in proc_inputs.items():
            if k not in ["input_ids", "attention_mask"]:
                batch_out[k] = v

        return batch_out


In [ ]:
import torch.nn as nn

class CADQwen2VL(nn.Module):
    def __init__(self, base_model, pc_token_encoder, tokenizer):
        super().__init__()
        self.base_model       = base_model          # Qwen2-VL with LoRA
        self.pc_token_encoder = pc_token_encoder    # Michelangelo+projector
        self.tokenizer        = tokenizer

        special_tokens = ["<PCD_START>", "<PCD_END>"]
        added = self.tokenizer.add_special_tokens(
            {"additional_special_tokens": special_tokens}
        )
        if added > 0:
            self.base_model.resize_token_embeddings(len(self.tokenizer))

        self.PCD_START_ID = self.tokenizer.convert_tokens_to_ids("<PCD_START>")
        self.PCD_END_ID   = self.tokenizer.convert_tokens_to_ids("<PCD_END>")

        self.embed_tokens = self.base_model.get_input_embeddings()

    def forward(
        self,
        input_ids,
        attention_mask,
        labels=None,
        points=None,
        feats=None,
        view_mask=None,
        pixel_values=None,
        **vision_kwargs,
    ):
        """
        During training, Trainer will pass:
          - input_ids, attention_mask, labels
          - points, feats, view_mask (from collator)
          - pixel_values (+ image_grid_thw, etc. inside **vision_kwargs)
        """
        device = input_ids.device
        B, L   = input_ids.shape

        # ---- 1) point cloud -> pc token ----
        if points is not None:
            pc_tokens = self.pc_token_encoder(
                points.to(device),
                feats.to(device),
                view_mask=view_mask.to(device),
            )  # [B,1,d]
            d_emb = self.embed_tokens.weight.dtype
            pc_tokens = pc_tokens.to(device=device, dtype=d_emb)
        else:
            pc_tokens = None

        # ---- 2) text embeddings ----
        text_embeds = self.embed_tokens(input_ids)  # [B,L,d]

        # ---- 3) prepend <PCD_START> [pc_token] <PCD_END> ----
        if pc_tokens is not None:
            pcd_start_ids = torch.full(
                (B, 1),
                self.PCD_START_ID,
                device=device,
                dtype=torch.long,
            )
            pcd_end_ids = torch.full(
                (B, 1),
                self.PCD_END_ID,
                device=device,
                dtype=torch.long,
            )

            pcd_start_emb = self.embed_tokens(pcd_start_ids)  # [B,1,d]
            pcd_end_emb   = self.embed_tokens(pcd_end_ids)    # [B,1,d]

            prefix_embeds = torch.cat([pcd_start_emb, pc_tokens, pcd_end_emb], dim=1)  # [B,3,d]
            full_embeds   = torch.cat([prefix_embeds, text_embeds], dim=1)             # [B,3+L,d]

            prefix_mask    = torch.ones(B, prefix_embeds.size(1),
                                        device=device, dtype=attention_mask.dtype)
            full_attn_mask = torch.cat([prefix_mask, attention_mask], dim=1)           # [B,3+L]
        else:
            full_embeds   = text_embeds
            full_attn_mask = attention_mask

        # ---- 4) extend labels to match new length ----
        if labels is not None:
            prefix_len = full_embeds.size(1) - L
            extended_labels = torch.full(
                (B, full_embeds.size(1)),
                -100,
                device=device,
                dtype=labels.dtype,
            )
            extended_labels[:, prefix_len:] = labels
        else:
            extended_labels = None

        # ---- 5) call Qwen2-VL base model ----
        outputs = self.base_model(
            inputs_embeds=full_embeds,
            attention_mask=full_attn_mask,
            labels=extended_labels,
            pixel_values=pixel_values,
            **vision_kwargs,  # e.g. image_grid_thw
        )
        return outputs


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from peft import LoraConfig, get_peft_model

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 1) Load Qwen2-VL and processor (skip if already loaded)
qwen_model_id = "Qwen/Qwen2-VL-2B-Instruct"   # or whatever you used
processor = Qwen2VLProcessor.from_pretrained(qwen_model_id)

base_model = Qwen2VLForConditionalGeneration.from_pretrained(
    qwen_model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
)
base_model.eval()

# 2) LoRA config for Qwen2-VL
target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",   # attention
    "gate_proj", "up_proj", "down_proj",      # MLP
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

qwen_lora = get_peft_model(base_model, lora_config)
qwen_lora.print_trainable_parameters()

# If you don't have pc_token_encoder yet, (re)create it:
pc_token_encoder = MichelangeloMultiViewTokens(
    base_encoder=michelangelo_encoder,
    projector=point_projector,
    freeze_base=True,   # or False if you want to fine-tune Michelangelo too
).to(device)

# 4) Build CADQwen2VL wrapper
cad_model = CADQwen2VL(
    base_model=qwen_lora,
    pc_token_encoder=pc_token_encoder,
    tokenizer=processor.tokenizer,
).to(device)

# 5) Freeze everything, then unfreeze LoRA + point projector
for p in cad_model.parameters():
    p.requires_grad = False

for name, p in cad_model.named_parameters():
    if "lora_" in name:  # LoRA modules
        p.requires_grad = True
    if "pc_token_encoder.projector" in name:  # point projector
        p.requires_grad = True

trainable = sum(p.numel() for p in cad_model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in cad_model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / {total/1e6:.2f}M")


In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
from transformers import TrainingArguments, Trainer

collator = CADTriModalCollatorRandomView(processor)

output_dir = "/content/drive/MyDrive/1_Diamond/CADVLM/qwen2vl_triple_lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=50,
    save_steps=50,
    save_total_limit=2,
    report_to=[],
    remove_unused_columns=False,   # important: don't drop our custom fields
    save_safetensors=False,
)

trainer = Trainer(
    model=cad_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collator,
)

trainer.train()


Step,Training Loss


In [ ]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa
from PIL import Image

INFER_PROMPT_TEMPLATE = (
    "You are a CAD program generator.\n"
    "Given a natural language description of a CAD part, "
    "generate a valid Omni-CAD JSON program that describes the part.\n\n"
    "Description:\n{description}\n\nJSON:\n"
)

def extract_json_block(text: str) -> str:
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        return text.strip()
    return text[start : end + 1].strip()


def run_val_inference_single_view(
    idx=None,
    max_new_tokens=512,
    n_points=2048,
    point_feats=3,
):
    """
    One-sample inference:
      text + ONE random image view + point cloud.
    Visualizes inputs and prints predicted JSON.
    """

    cad_model.eval()

    # ----- 1. pick a val example -----
    if idx is None:
        idx = random.randint(0, len(val_dataset) - 1)

    ex = val_dataset[idx]
    desc        = ex["description"]
    json_text   = ex["json_text"]
    image_paths = ex["image_paths"]
    point_paths = ex["point_paths"]

    print(f"\n=== Tri-modal inference (single view) on val idx = {idx} ===\n")
    print("Description:\n", desc)
    print("\nGround-truth JSON snippet:\n", json_text[:600], "...\n")

    # ----- 2. visualize text -----
    plt.figure(figsize=(10, 4))
    plt.axis("off")
    plt.title("Input description + GT JSON snippet")
    snippet = json_text[:400]
    txt_block = f"Description:\n{desc}\n\nGT JSON (truncated):\n{snippet}"
    plt.text(
        0.01,
        0.99,
        txt_block,
        va="top",
        ha="left",
        fontsize=8,
        family="monospace",
        wrap=True,
    )
    plt.show()

    # ----- 3. choose ONE random image view -----
    if len(image_paths) == 0:
        print("No images for this example; aborting.")
        return

    img_path = random.choice(image_paths)
    print("[IMG] using single view:", img_path)
    image = Image.open(img_path).convert("RGB")

    plt.figure(figsize=(4, 4))
    plt.imshow(image)
    plt.axis("off")
    plt.title("Random chosen view")
    plt.show()

    # ----- 4. load + plot point cloud -----
    if len(point_paths) == 0:
        print("No pointcloud for this example; aborting.")
        return

    pc_path = point_paths[0]
    print("[PC] using pointcloud:", pc_path)

    pts = load_pointcloud_simple(pc_path, n_points=n_points)  # [N,3]
    pts_plot = pts
    if pts_plot.shape[0] > 5000:
        idx_vis = np.random.choice(pts_plot.shape[0], 5000, replace=False)
        pts_plot = pts_plot[idx_vis]

    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(
        pts_plot[:, 0],
        pts_plot[:, 1],
        pts_plot[:, 2],
        s=1,
    )
    ax.set_title("Input point cloud")
    plt.show()

    # ----- 5. build Qwen2-VL inputs: text + 1 image -----
    prompt = INFER_PROMPT_TEMPLATE.format(description=desc)

    messages = [{
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": prompt},
        ],
    }]

    chat_prompt = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    proc_inputs = processor(
        text=[chat_prompt],
        images=[image],   # single PIL image in a list
        return_tensors="pt",
    ).to(device)

    input_ids      = proc_inputs["input_ids"]
    attention_mask = proc_inputs["attention_mask"]
    vision_kwargs = {
        k: v for k, v in proc_inputs.items()
        if k not in ["input_ids", "attention_mask"]
    }

    # ----- 6. prepare pc token -----
    pts_torch = torch.from_numpy(pts).to(device)      # [N,3]
    pts_torch = pts_torch.unsqueeze(0).unsqueeze(0)   # [1,1,N,3]
    feats     = torch.zeros(1, 1, n_points, point_feats,
                            dtype=torch.float32, device=device)
    view_mask = torch.ones(1, 1, dtype=torch.bool, device=device)

    embed_tokens = cad_model.embed_tokens
    B, L = input_ids.shape

    text_embeds = embed_tokens(input_ids)  # [1,L,d]

    with torch.no_grad():
        pc_tokens = cad_model.pc_token_encoder(
            pts_torch,
            feats,
            view_mask=view_mask,
        )  # [1,1,d]
        pc_tokens = pc_tokens.to(
            device=device,
            dtype=embed_tokens.weight.dtype,
        )

    pcd_start_ids = torch.full(
        (B, 1),
        cad_model.PCD_START_ID,
        device=device,
        dtype=torch.long,
    )
    pcd_end_ids = torch.full(
        (B, 1),
        cad_model.PCD_END_ID,
        device=device,
        dtype=torch.long,
    )

    pcd_start_emb = embed_tokens(pcd_start_ids)
    pcd_end_emb   = embed_tokens(pcd_end_ids)

    prefix_embeds = torch.cat(
        [pcd_start_emb, pc_tokens, pcd_end_emb], dim=1
    )  # [1,3,d]
    full_embeds = torch.cat(
        [prefix_embeds, text_embeds], dim=1
    )  # [1,3+L,d]

    prefix_mask    = torch.ones(B, prefix_embeds.size(1),
                                device=device,
                                dtype=attention_mask.dtype)
    full_attn_mask = torch.cat([prefix_mask, attention_mask], dim=1)  # [1,3+L]

    # ----- 7. generate JSON -----
    with torch.no_grad():
        out_ids = cad_model.base_model.generate(
            inputs_embeds=full_embeds,
            attention_mask=full_attn_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            **vision_kwargs,
        )

    out_text = processor.tokenizer.decode(out_ids[0], skip_special_tokens=True)
    pred_json = extract_json_block(out_text)

    # ----- 8. show predicted JSON -----
    print("\nPredicted JSON (truncated):\n")
    print(pred_json[:2000], "...\n")

    plt.figure(figsize=(10, 6))
    plt.axis("off")
    plt.title("Predicted JSON (first ~2000 chars)")
    plt.text(
        0.01,
        0.99,
        pred_json[:2000],
        va="top",
        ha="left",
        fontsize=8,
        family="monospace",
        wrap=True,
    )
    plt.show()

    return {
        "idx": idx,
        "description": desc,
        "gt_json": json_text,
        "pred_json": pred_json,
        "image_path": img_path,
        "point_path": pc_path,
    }


In [ ]:
_ = run_val_inference_single_view(idx=2006)